In [ ]:
import os
os.path.abspath("")

In [ ]:
import os, sys
from loguru import logger

LOG_ROOT = os.path.abspath("./")
LOG_FILE = LOG_ROOT + "/logs/sr-3.log"

# Remove all handlers and reset stderr
logger.remove(handler_id=None)
logger.add(
    LOG_FILE,
    level="TRACE",
    mode="w",
    backtrace=False,
    diagnose=True,
    colorize=False,
    format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}",
)
logger.info("\r\n" + ">" * 29 + "\r\n" + ">>> New execution started >>>" + "\r\n" + ">" * 29)
# To filter log level: TRACE=5, DEBUG=10, INFO=20, SUCCESS=25, WARNING=30, ERROR=40, CRITICAL=50
logger.add(sys.stdout, level="INFO", filter=lambda record: record["level"].no < 40, colorize=True)
logger.add(sys.stderr, level="ERROR", backtrace=False, diagnose=True, colorize=True)

## Load data

In [1]:
import pandas as pd
REPORT_PATH = "/home/yuxiangliao/PhD/data/mimic_cxr_reports_core.json"
df = pd.read_json(REPORT_PATH,orient="records",lines=True)
print(df)

id_list = df.loc[:,'sid'].to_list()
findings_list = df.loc[:,'findings'].to_list()
impression_list = df.loc[:,'impression'].to_list()
pfi_list = df.loc[:,'provisional_findings_impression'].to_list()
fai_list = df.loc[:,'findings_and_impression'].to_list()

DATA_SIZE = len(id_list)

              pid        sid  \
0       p10000032  s50414267   
1       p10000032  s53189527   
2       p10000032  s53911762   
3       p10000032  s56699142   
4       p10000764  s57375967   
...           ...        ...   
227830  p19999442  s58708861   
227831  p19999733  s57132437   
227832  p19999987  s55368167   
227833  p19999987  s58621812   
227834  p19999987  s58971208   

                                                 findings  \
0       There is no focal consolidation, pleural effus...   
1       The cardiac, mediastinal and hilar contours ar...   
2       Single frontal view of the chest provided. \n ...   
3       The lungs are clear of focal consolidation, pl...   
4       PA and lateral views of the chest provided.   ...   
...                                                   ...   
227830  ET tube ends 4.7 cm above the carina.  NG tube...   
227831  The lungs are clear, and the cardiomediastinal...   
227832  There has been interval extubation and improve...   
22783

# Check CoreNLP

In [ ]:
!ls $CORENLP_HOME

# CoreNLP

In [2]:
import stanza
from stanza.server import CoreNLPClient

In [6]:
import time
CUSTOM_PROPS = {
    'annotators':'tokenize, ssplit, pos, lemma, ner, depparse, coref',
    "coref.algorithm": "statistical"
}
with CoreNLPClient(memory='8G', threads=16, endpoint='http://localhost:8801', be_quiet=False, 
                   properties=CUSTOM_PROPS) as client:
    start = time.time()
    for text in findings_list[0:100]:
        document = client.annotate(text)
    done = time.time()
    elapsed = done - start
    print(elapsed)

2022-07-12 20:02:29 INFO: Writing properties to tmp file: corenlp_server-2f7f43d6a526462e.props
2022-07-12 20:02:29 INFO: Starting server with command: java -Xmx5G -cp /home/yuxiangliao/PhD/Stanford_CoreNLP/stanford-corenlp-4.4.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 8801 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-2f7f43d6a526462e.props -preload -outputFormat serialized


25.239668130874634


Define connection

Start client

Run

Stop client

# Stanza

Biomedical models

In [ ]:
import stanza
stanza.download('en', package='mimic')
stanza.download('en', package='radiology')

Stanza processor: https://stanfordnlp.github.io/stanza/pipeline.html#processors

In [7]:
from stanza.pipeline.core import DownloadMethod
processor_dict = {
    'tokenize': 'mimic', 
    'pos': 'mimic', 
    'lemma': 'mimic',
    'depparse': 'mimic',
    # 'sentiment':'sstplus', # Sentiment scores of 0, 1, or 2 (negative, neutral, positive).
    'constituency': 'wsj', # wsj, wsj_bert, wsj_roberta
    'ner': 'radiology',
}
nlp = stanza.Pipeline('en', processors=processor_dict, package=None, 
                      download_method=DownloadMethod.REUSE_RESOURCES,
                      verbose=False) # logging_level='WARN'
doc = nlp(findings_list[8690])
# print out dependency tree
print(doc)

[
  [
    {
      "id": 1,
      "text": "Compared",
      "lemma": "compare",
      "upos": "VERB",
      "xpos": "VBN",
      "head": 5,
      "deprel": "case",
      "start_char": 0,
      "end_char": 8,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 2,
      "text": "with",
      "lemma": "with",
      "upos": "ADP",
      "xpos": "IN",
      "head": 5,
      "deprel": "case",
      "start_char": 9,
      "end_char": 13,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 3,
      "text": "the",
      "lemma": "the",
      "upos": "DET",
      "xpos": "DT",
      "head": 5,
      "deprel": "det",
      "start_char": 14,
      "end_char": 17,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "prior",
      "lemma": "prior",
      "upos": "ADJ",
      "xpos": "JJ",
      "head": 5,
      "deprel": "amod",
      "start_char": 18,
      "end_char": 23,
      "ner": "O

Processing Multiple Documents

28s -> 100docs

In [9]:
import stanza
processor_dict = {
    'tokenize': 'mimic', 
    'pos': 'mimic', 
    'lemma': 'mimic',
    'depparse': 'mimic',
    # 'sentiment':'sstplus', # Sentiment scores of 0, 1, or 2 (negative, neutral, positive).
    'constituency': 'wsj', # wsj, wsj_bert, wsj_roberta
    'ner': 'radiology',
}
nlp = stanza.Pipeline('en', processors=processor_dict, package=None, 
                      download_method=DownloadMethod.REUSE_RESOURCES,
                      verbose=False) # logging_level='WARN'
documents = findings_list[0:300] # Documents that we are going to process
in_docs = [stanza.Document([], text=d) for d in documents] # Wrap each document with a stanza.Document object
out_docs = nlp(in_docs) # Call the neural pipeline on this list of documents
print(out_docs[1]) # The output is also a list of stanza.Document objects, each output corresponding to an input Document object

[
  [
    {
      "id": 1,
      "text": "The",
      "lemma": "the",
      "upos": "DET",
      "xpos": "DT",
      "head": 7,
      "deprel": "det",
      "start_char": 0,
      "end_char": 3,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 2,
      "text": "cardiac",
      "lemma": "cardiac",
      "upos": "NOUN",
      "xpos": "NN",
      "head": 9,
      "deprel": "nsubj",
      "start_char": 4,
      "end_char": 11,
      "ner": "S-ANATOMY",
      "multi_ner": [
        "S-ANATOMY"
      ]
    },
    {
      "id": 3,
      "text": ",",
      "lemma": ",",
      "upos": "PUNCT",
      "xpos": ",",
      "head": 4,
      "deprel": "punct",
      "start_char": 11,
      "end_char": 12,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "mediastinal",
      "lemma": "mediastinal",
      "upos": "ADJ",
      "xpos": "JJ",
      "head": 2,
      "deprel": "conj",
      "start_char": 13,
      "end_cha